In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy
import matplotlib.pyplot as plt


from sodapy import Socrata
from config import API_Token

plt.style.use('fivethirtyeight')

%matplotlib inline

## Section 1: Load and First Glance
1. load data through API with preliminary filter
2. dataframe general information
3. adjust data types

### Assumptions:
large dataset, only query calendar year and 5 years from 2018 to 2022

In [23]:
# API Call
socrata_domain = 'data.sfgov.org,'
socrata_dataset_identifier = '88g8-5mnd'
client = Socrata("data.sfgov.org",API_Token)
# add filter
results = client.get_all(socrata_dataset_identifier, 
    where = "year in ('2018','2019','2020','2021','2022') and year_type = 'Calendar'" )

df_origin = pd.DataFrame(results)
# alternative#1 csv file
# df_origin = pd.read_csv('data/Employee_Compensation.csv')
# alternative#2 json file
# df_origin = pd.read_json('https://data.sfgov.org/resource/88g8-5mnd.json')

df_origin.shape  # (168437, 22)

(168437, 22)


,organization_group_code,job_family_code,job_code,year_type,year,organization_group,department_code,department,union_code,union,...,employee_identifier,salaries,overtime,other_salaries,total_salary,retirement,health_and_dental,other_benefits,total_benefits,total_compensation
0,01,1000,1032,Calendar,2018,Public Protection,ADP,Adult Probation,021,"Prof & Tech Eng, Local 21",...,38405932,81096.04,0,0,81096.04,15556.78,14433.94,6076.79,36067.51,117163.55
1,01,1000,1041,Calendar,2018,Public Protection,ADP,Adult Probation,021,"Prof & Tech Eng, Local 21",...,37915952,115053.11,0,0,115053.11,22071.21,14433.94,9059.88,45565.03,160618.14
2,01,1000,1053,Calendar,2018,Public Protection,ADP,Adult Probation,021,"Prof & Tech Eng, Local 21",...,37905927,107355.32,0,0,107355.32,20594.4,14433.94,8171.53,43199.87,150555.19
3,01,1000,1054,Calendar,2018,Public Protection,ADP,Adult Probation,021,"Prof & Tech Eng, Local 21",...,37477463,154758,0,0,154758,28939.26,14433.94,10980.3,54353.5,209111.5
4,01,1000,1062,Calendar,2018,Public Protection,ADP,Adult Probation,021,"Prof & Tech Eng, Local 21",...,37483360,91988.81,0,0,91988.81,17647.11,14433.94,7214.21,39295.26,131284.07


In [ ]:
df = df_origin.copy()
df.head(5)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168437 entries, 0 to 168436
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   organization_group_code  168437 non-null  object
 1   job_family_code          168437 non-null  object
 2   job_code                 168437 non-null  object
 3   year_type                168437 non-null  object
 4   year                     168437 non-null  object
 5   organization_group       168437 non-null  object
 6   department_code          168437 non-null  object
 7   department               168437 non-null  object
 8   union_code               168437 non-null  object
 9   union                    168437 non-null  object
 10  job_family               168437 non-null  object
 11  job                      168437 non-null  object
 12  employee_identifier      168437 non-null  object
 13  salaries                 168437 non-null  object
 14  overtime            

## transform

In [27]:
# clean up column names, save into a list, check the data type
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
column_name = df.columns.tolist()
print(column_name)
print(f'We have total of {len(column_name)} columns')
print(df.dtypes)

['organization_group_code', 'job_family_code', 'job_code', 'year_type', 'year', 'organization_group', 'department_code', 'department', 'union_code', 'union', 'job_family', 'job', 'employee_identifier', 'salaries', 'overtime', 'other_salaries', 'total_salary', 'retirement', 'health_and_dental', 'other_benefits', 'total_benefits', 'total_compensation']
We have total of 22 columns
organization_group_code    object
job_family_code            object
job_code                   object
year_type                  object
year                       object
organization_group         object
department_code            object
department                 object
union_code                 object
union                      object
job_family                 object
job                        object
employee_identifier        object
salaries                   object
overtime                   object
other_salaries             object
total_salary               object
retirement                 object
health_

### asumption: 
Transform the year and 10 salaries-related columns into numerical, and keep information columns such as group code, familiy code as categorical.

In [36]:
# define the numerical column
numerical_column_names = [ 'year',  'employee_identifier', 'salaries', 'overtime', 'other_salaries', 'total_salary', 'retirement', 'health_and_dental', 'other_benefits', 'total_benefits', 'total_compensation']

# change data types
for column in column_name:
    if column in numerical_column_names:
        if column == 'year':
            df[column] = df[column].astype('int')
        else:
            df[column] = df[column].astype('float64')
print(df.dtypes)

organization_group_code     object
job_family_code             object
job_code                    object
year_type                   object
year                         int64
organization_group          object
department_code             object
department                  object
union_code                  object
union                       object
job_family                  object
job                         object
employee_identifier        float64
salaries                   float64
overtime                   float64
other_salaries             float64
total_salary               float64
retirement                 float64
health_and_dental          float64
other_benefits             float64
total_benefits             float64
total_compensation         float64
dtype: object


In [38]:
df.head()

,organization_group_code,job_family_code,job_code,year_type,year,organization_group,department_code,department,union_code,union,...,employee_identifier,salaries,overtime,other_salaries,total_salary,retirement,health_and_dental,other_benefits,total_benefits,total_compensation
0,01,1000,1032,Calendar,2018,Public Protection,ADP,Adult Probation,021,"Prof & Tech Eng, Local 21",...,38405932.0,81096.04,0.0,0.0,81096.04,15556.78,14433.94,6076.79,36067.51,117163.55
1,01,1000,1041,Calendar,2018,Public Protection,ADP,Adult Probation,021,"Prof & Tech Eng, Local 21",...,37915952.0,115053.11,0.0,0.0,115053.11,22071.21,14433.94,9059.88,45565.03,160618.14
2,01,1000,1053,Calendar,2018,Public Protection,ADP,Adult Probation,021,"Prof & Tech Eng, Local 21",...,37905927.0,107355.32,0.0,0.0,107355.32,20594.40,14433.94,8171.53,43199.87,150555.19
3,01,1000,1054,Calendar,2018,Public Protection,ADP,Adult Probation,021,"Prof & Tech Eng, Local 21",...,37477463.0,154758.00,0.0,0.0,154758.00,28939.26,14433.94,10980.30,54353.50,209111.50
4,01,1000,1062,Calendar,2018,Public Protection,ADP,Adult Probation,021,"Prof & Tech Eng, Local 21",...,37483360.0,91988.81,0.0,0.0,91988.81,17647.11,14433.94,7214.21,39295.26,131284.07


# eda
1. Summary Statistics
2. miss value
3. value counts

In [37]:
df.describe()

,year,employee_identifier,salaries,overtime,other_salaries,total_salary,retirement,health_and_dental,other_benefits,total_benefits,total_compensation
count,168437.000000,1.684370e+05,168437.000000,168437.000000,168437.000000,168437.000000,168437.000000,168437.000000,168437.000000,168437.000000,168437.000000
mean,2019.465628,4.230149e+07,82429.954530,7078.091892,3910.599044,93418.645465,17197.856361,11901.419924,5816.690627,34915.966912,128334.612378
std,1.110878,3.691154e+06,51999.156269,16048.602443,7786.099876,60913.718752,12232.590140,6681.298768,3702.460204,20464.999950,79653.803756
min,2018.000000,0.000000e+00,-9942.330000,-22453.280000,-480.000000,0.000000,-37.890000,-745.720000,0.000000,0.000000,0.000000
25%,2018.000000,3.883603e+07,47643.000000,0.000000,0.000000,52245.360000,8876.520000,7718.990000,2528.620000,20528.870000,74311.860000
50%,2019.000000,4.183610e+07,80493.500000,62.070000,862.500000,89841.010000,16923.670000,14950.300000,5923.960000,38512.260000,128503.140000
75%,2020.000000,4.542635e+07,115906.160000,6562.110000,4581.330000,130409.340000,24197.040000,16303.470000,8699.000000,47968.360000,179060.980000
max,2021.000000,4.933681e+07,596615.860000,315896.410000,326959.390000,596615.860000,150142.960000,59405.230000,18779.490000,175232.200000,762684.040000


## insights:
the year range is within range, 2022 data shifts 50% median value to 2019, instead of 2020

salaries are righ-skewed(mean > median), since large value/outlier pull mean towards right.

6 unique organization group_code, 51 department_code, 57 job_family_code 56 family, 1111 job_code, 66 or 69 union 


### investigation:
1. negative in salaries to make high level total_salary as 0, will try to take a closely look at these 0 rows
2. outlier

In [29]:
df.describe(include='object')


,organization_group_code,job_family_code,job_code,year_type,organization_group,department_code,department,union_code,union,job_family,job
count,168437,168437,168437,168437,168437,168437,168437,168437,168437,168437,168437
unique,6,57,1111,1,6,51,51,69,66,56,1111
top,02,2300,9163,Calendar,"Public Works, Transportation & Commerce",DPH,Public Health,790,"SEIU, Local 1021, Misc",Nursing,Transit Operator
freq,53192,18130,10986,168437,53192,38362,38362,46720,63754,18130,10986


In [40]:
df.isnull().sum()

,organization_group_code,job_family_code,job_code,year_type,year,organization_group,department_code,department,union_code,union,...,employee_identifier,salaries,overtime,other_salaries,total_salary,retirement,health_and_dental,other_benefits,total_benefits,total_compensation
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
168433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
168434,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
168435,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [42]:
# no missing data
miss_percent = df.isnull().sum()/ df.shape[0] * 100
miss_percent

organization_group_code    0.0
job_family_code            0.0
job_code                   0.0
year_type                  0.0
year                       0.0
organization_group         0.0
department_code            0.0
department                 0.0
union_code                 0.0
union                      0.0
job_family                 0.0
job                        0.0
employee_identifier        0.0
salaries                   0.0
overtime                   0.0
other_salaries             0.0
total_salary               0.0
retirement                 0.0
health_and_dental          0.0
other_benefits             0.0
total_benefits             0.0
total_compensation         0.0
dtype: float64

Section 1 conclusion:

We have total of 22 columns


In [6]:
test_df = df.groupby(['Year Type','Year','Job'])['Salaries','Total Compensation'].mean()
test_df.head(10)

/var/folders/30/kt35bq3n3_b90dxt80z_vjjw0000gn/T/ipykernel_41980/849934878.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  test_df = df.groupby(['Year Type','Year','Job'])['Salaries','Total Compensation'].mean()


Salaries  Total Compensation
Year Type Year Job                                                        
Calendar  2013 Account Clerk              40388.399216        61702.154608
               Accountant I               67838.413333        99504.836667
               Accountant II              57121.076629        83004.125506
               Accountant III             67190.154862        94839.112844
               Accountant IV              85711.871500       118490.453333
               Accountant Intern          24530.535614        36561.595965
               Acupuncturist              75411.000000       111214.280000
               Adm, SFGH Medical Center  276831.500000       355107.360000
               Admin Analyst 3            68009.620000        96174.406667
               Administrative Analyst     62668.591644        89979.665274

In [8]:
test_df = df.groupby(['Year Type','Year'])[['Salaries','Total Compensation']].mean()
test_df.head(10)

Salaries  Total Compensation
Year Type Year                                  
Calendar  2013  64717.640360        99638.412577
          2014  62261.763051        97717.615926
          2015  63801.845319        98605.356720
          2016  66498.335433       102014.919491
          2017  74770.937352       114479.846099
          2018  76919.412119       118348.406340
          2019  77060.393590       119842.301509
          2020  86018.214281       134258.764395
          2021  90601.774692       142360.177374
Fiscal    2013  62875.983167        96087.879968